### Install dependencies

In [ ]:
# ! pip3 install selenium
# brew install geckodriver

### [TODO] Programatically create new profiles. 
#### 1. Borrow code from OpenWPM: https://github.com/mozilla/OpenWPM/blob/9b4d3582bc3bdb4e319874a427d63ff14feadd2c/openwpm/deploy_browsers/deploy_firefox.py

### If unexpected errors occur, try changing marionette port

In [1]:
from selenium import webdriver
import time

from selenium.webdriver import FirefoxOptions
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

### Manually updated preferences in profile (Important!):
```
1. Select custom privacy protection and disable all
2. Disable auto updates
3. Disable default browser check
4. Disable recommended by pocket 
```
------
```
1. extensions.experiments.enabled --> True
2. xpinstall.signatures.required --> False
3. network.cookie.cookieBehavior --> 0
4. browser.shell.checkDefaultBrowser --> False
```

### Helper functions

In [2]:
FIREFOX_EXE_PATH = '/Applications/Nightly.app/Contents/MacOS/firefox-bin'
GECKO_PATH = '/opt/homebrew/bin/geckodriver'
EXTENSION_PATH = '/Users/umariqbal/Documents/voice-assistant-central/browser-extension.zip'

In [37]:
def get_webdriver(profile_path, gecko_path, exe_path, ext_loc):
    options = FirefoxOptions()
    options.add_argument('-profile')
    options.add_argument(profile_path)
    
    firefox_capabilities = DesiredCapabilities.FIREFOX
    firefox_capabilities['marionette'] = True

    
    firefox_profile = webdriver.FirefoxProfile()
    firefox_bin = FirefoxBinary(firefox_path=exe_path)
    driver = webdriver.Firefox(firefox_binary= firefox_bin,
                               capabilities=firefox_capabilities, 
                               options=options, 
                               executable_path=gecko_path, 
                               service_args=["--marionette-port", "2828"])

    driver.install_addon(ext_loc, temporary=True)
    
    return driver

def quit_driver(driver):
    try:
        driver.quit()
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        
def signin(driver, singin_page, email, password):
    try:
        driver.get(singin_page)
        time.sleep(3)

        signin_button = driver.find_element_by_id('a-autoid-0-announce')
        signin_button.click()
        time.sleep(3)
        
        try:
            email_input = driver.find_element_by_id('ap_email')

        except Exception:
            ## Assume that the user is already signed in 
            print('Already signed in. Returning.')
            return True
            
        email_input.send_keys(email)
        time.sleep(3)
        
        continue_button = driver.find_element_by_id('continue')
        continue_button.click()
        time.sleep(3)
        
        password_input = driver.find_element_by_id('ap_password')
        password_input.send_keys(password)
        time.sleep(3)
        
        signin_button = driver.find_element_by_id('signInSubmit')
        signin_button.click()
        time.sleep(3)
        
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        return False
    
    return True


def install_skill(driver, url, perm = False):
    try:
        driver.get(url)
        time.sleep(3)
        
        skill_enable_button = driver.find_element_by_id("a2s-skill-enable-button")
        skill_enable_button.click()
        time.sleep(3)
        
        if perm:
            try 
                permission_button = driver.find_element_by_xpath("//input[@type='submit'][@class='a-button-input'][not(@aria-labelledby)]")
                permission_button.click()
                time.sleep(3)
       
            except Exception:
                print('Something went wrong when enabling permissions')
                return False, 'perm'
            

    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        return False, 'sim'
    
    return True, 'sim'

In [9]:
def train_persona(profile_path, signin_page, email, password, skills_urls, 
                  gecko_path=GECKO_PATH, exe_path=FIREFOX_EXE_PATH, ext_loc=EXTENSION_PATH):
    
    driver = get_webdriver(profile_path, gecko_path, exe_path, ext_loc)
    print('WebDriver registered')
    
    signin_status = signin(driver, signin_page, email, password)

    total_installed = 0
    if signin_status:
        print('Signed in.')
        for skill_url in skills_urls:
            install_status = install_skill(driver, skill_url):
            # Log fully installed and partially installed extensions
            if install_status[0]:
                total_installed += 1
    else:
        print('Could not sign in. Stopping the process')
        quit_driver(driver)
        
    
    quit_driver(driver)
    print('Installed %f skills.', (total_installed / len(skills_urls)) * 100.0)

In [11]:
# profile_path = '/Users/umariqbal/Library/Application Support/Firefox/Profiles/y7zzv5nf.va-test'
profile_path = '/Users/umariqbal/Library/Application Support/Firefox/Profiles/nf5umchy.va-night-test'
signin_page = 'https://www.amazon.com/Dominos-Pizza-LLC/dp/B01B5G99CC/ref=lp_14284822011_1_7?s=digital-skills&ie=UTF8'

email = 'phillip-phillips-1@protonmail.com'
password = 'change.me'


train_persona(profile_path, signin_page, email, password, 
                  gecko_path=GECKO_PATH, exe_path=FIREFOX_EXE_PATH, ext_loc=EXTENSION_PATH)

In [25]:
install_skill(driver, skill_url)

Something went wrong:  Message: Unable to locate element: /html/body/div[4]/div/div/div[2]/span/span/span/input



False

In [29]:
# skill_enable_button = driver.find_element_by_id("a2s-skill-enable-button")
# skill_enable_button.click()
# time.sleep(3)
        
# driver.find_element_by_xpath("/html/body/div[4]/div/div/div[2]/span/span/span/input")
permission_button = driver.find_element_by_xpath("/html/body/div[4]/div/div/div[2]/span/span/span/input")

In [33]:
permission_button.click()
# driver.quit()

In [38]:
gecko_path=GECKO_PATH
exe_path=FIREFOX_EXE_PATH
ext_loc=EXTENSION_PATH

skills_urls = [signin_page]

driver = get_webdriver(profile_path, gecko_path, exe_path, ext_loc)
print('WebDriver registered')

signin_status = signin(driver, signin_page, email, password)

total_installed = 0
if signin_status:
    print('Signed in.')
    for skill_url in skills_urls:
        if install_skill(driver, skill_url):
            total_installed += 1
else:
    print('Could not sign in. Stopping the process')
    quit_driver(driver)

print(f'Installed %0.2f skills.' % ((total_installed / len(skills_urls)) * 100.0))

WebDriver obtained
Already signed in. Returning.
Signed in.
Installed %f skills. 100.0


In [36]:
quit_driver(driver)

In [ ]:
def train_persona(profile_path, gecko_path, exe_path, ext_loc, webpage_to_open):
    options = FirefoxOptions()
    options.add_argument('-profile')
    options.add_argument(profile_path)
    
    firefox_capabilities = DesiredCapabilities.FIREFOX
    firefox_capabilities['marionette'] = True

    
    firefox_profile = webdriver.FirefoxProfile()
    firefox_bin = FirefoxBinary(firefox_path=exe_path)
    driver = webdriver.Firefox(firefox_binary= firefox_bin,
                               capabilities=firefox_capabilities, 
                               options=options, 
                               executable_path=gecko_path, 
                               service_args=["--marionette-port", "2828"])

    driver.install_addon(ext_loc, temporary=True)

    try:
        driver.get(webpage_to_open)
        time.sleep(30)
        
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        pass

    finally:
        driver.quit()

In [ ]:
def signin(singin_page):
    #     firefox_options = webdriver.FirefoxOptions()
    #     firefox_options.add_argument('--disable-application-cache')
    #     firefox_options.add_argument('--disable-infobars')
    #     firefox_options.add_argument('--no-sandbox')    
    #     firefox_options.add_argument('--chrome-binary=' + binary_path)
    #     firefox_options.binary_location = binary_path
    driver = webdriver.Firefox()
    
    try:
        driver.get(singin_page)
        time.sleep(3)

        signin_button = driver.find_element_by_id('a-autoid-0-announce')
        signin_button.click()
        time.sleep(3)
        
        email_input = driver.find_element_by_id('ap_email')
        email_input.send_keys('phillip-phillips-1@protonmail.com')
        time.sleep(3)
        
        continue_button = driver.find_element_by_id('continue')
        continue_button.click()
        time.sleep(3)
        
        
        password_input = driver.find_element_by_id('ap_password')
        password_input.send_keys('change.me')
        time.sleep(3)
        
        signin_button = driver.find_element_by_id('signInSubmit')
        signin_button.click()
        time.sleep(3)
        
        skill_enable_button = driver.find_element_by_id("a2s-skill-enable-button")
        skill_enable_button.click()
        time.sleep(3)
        
        permission_button = driver.find_element_by_id("a-button-input")
        permission_button.click()
        time.sleep(10)
        
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        pass

    finally:
        driver.quit()
    

def crawl___(webpage_to_open):
#     firefox_options = webdriver.FirefoxOptions()
#     firefox_options.add_argument('--disable-application-cache')
#     firefox_options.add_argument('--disable-infobars')
#     firefox_options.add_argument('--no-sandbox')    
#     firefox_options.add_argument('--chrome-binary=' + binary_path)
#     firefox_options.binary_location = binary_path
    driver = webdriver.Firefox()
    
    try:
        driver.get(webpage_to_open)
        time.sleep(3)
        skill_enable_button = driver.find_element_by_id("a2s-skill-enable-button")
        
        skill_enable_button.click()
        time.sleep(10)
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        pass

    finally:
        driver.quit()

In [ ]:
signin('https://www.amazon.com/Dominos-Pizza-LLC/dp/B01B5G99CC/ref=lp_14284822011_1_7?s=digital-skills&ie=UTF8')
# crawl('https://www.amazon.com/Dominos-Pizza-LLC/dp/B01B5G99CC/ref=lp_14284822011_1_7?s=digital-skills&ie=UTF8')

### Rough work

In [ ]:
# def crawl__(profile_path, webpage_to_open):
#     options = Options()
#     options.add_argument("-profile")
#     options.add_argument(profile_path)
#     firefox_capabilities = DesiredCapabilities.FIREFOX
#     firefox_capabilities['marionette'] = True
#     driver = webdriver.Firefox(capabilities=firefox_capabilities, firefox_options=options)
    
#     driver.get(webpage_to_open)
#     time.sleep(3)

# from selenium.webdriver.firefox.options import Options
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

# options = Options()
# options.add_argument("-profile")
# options.add_argument("/Users/umariqbal/Library/Application Support/Firefox/Profiles/y7zzv5nf.va-test")
# firefox_capabilities = DesiredCapabilities.FIREFOX
# firefox_capabilities['marionette'] = True
# driver = webdriver.Firefox(capabilities=firefox_capabilities, firefox_options=options, service_args=["--marionette-port", "2828"])
# driver.get('https://www.cnn.com')


# crawl__(profile_path, 'cnn.com')

# from selenium import webdriver
# fp = webdriver.FirefoxProfile('/Users/umariqbal/Library/Application Support/Firefox/Profiles/y7zzv5nf.va-test')
# driver = webdriver.Firefox(fp)
# driver.get('https://www.cnn.com')

# override_webdriver = """script = document.createElement("script"); 
#     script.type = 'text/javascript'; 
#     script.id = "NotVeryUniqueID"; 
#     script.innerHTML = "Object.defineProperty(navigator, 'webdriver', {get: () => false}); document.getElementById('scripttttttt').remove();"; 
#     document.documentElement.prepend(script);"""

#     options.set_preference('dom.webdriver.enabled', False)
#     options.set_preference('useAutomationExtension', False)
    
#     options.set_preference('extensions.experiments.enabled', True)    
#     options.set_preference('xpinstall.signatures.required', False)
#     firefox_profile.update_preferences()

In [ ]:
# crawl(profile_path, gecko_path, exe_path, ext_loc, 'https://www.cnn.com')

# driver = get_webdriver(profile_path, gecko_path, exe_path, ext_loc)
# signin_status = signin(driver, signin_page)

# if signin_status:
#     install_skills()
# else:
#     print('Could not sign in succesfully!')
# #     driver.quit()
